In [ ]:
# Created on April 25, 2025
# Built from web crawler developed by Bo Zhao (see https://github.com/jakobzhao/web-crawler)
# Developed in Google Colab

# @author: Alex Kirchmeier
# @description: Practice web crawler that collects article titles from https://www.theguardian.com/us

In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

Executing: /tmp/apt-key-gpghome.7gztnaEWrG/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.EAYWsugDpz/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.MnjeRngGtT/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [ ]:
!apt-get update
!apt-get install chromium chromium-driver
!pip3 install selenium
!pip install kora -q

Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian buster-updates InRelease [56.6 kB]
Get:3 http://deb.debian.org/debian-security buster/updates InRelease [34.8 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:9 http://deb.debian.org/debian buster/main amd64 Packages [10.7 MB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 http://deb.debian.org/debian buster-updates/main amd64 Packages [9,745 B]
Get:13 http://deb.debian.org/debian-security buster/updates/main amd64 Packages [796 kB]
Hit:14 https://

The following three cells configure Chromedriver to ensure that all features are compatible.

In [ ]:
# Remove the Old Chrome and ChromeDriver

!apt-get purge chromium-browser chromium-chromedriver

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Package 'chromium-browser' is not installed, so not removed
Package 'chromium-chromedriver' is not installed, so not removed
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [ ]:
# Install a Matching Version of Chrome and ChromeDriver

# Download Chrome v90
!wget https://dl.google.com/linux/chrome/deb/pool/main/g/google-chrome-stable/google-chrome-stable_90.0.4430.212-1_amd64.deb
!dpkg -i google-chrome-stable_90.0.4430.212-1_amd64.deb

# Install ChromeDriver for Chrome v90
!wget https://chromedriver.storage.googleapis.com/90.0.4430.24/chromedriver_linux64.zip
!unzip chromedriver_linux64.zip -d /usr/bin/

--2025-04-26 03:42:47--  https://dl.google.com/linux/chrome/deb/pool/main/g/google-chrome-stable/google-chrome-stable_90.0.4430.212-1_amd64.deb
Resolving dl.google.com (dl.google.com)... 142.250.98.93, 142.250.98.190, 142.250.98.136, ...
Connecting to dl.google.com (dl.google.com)|142.250.98.93|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-04-26 03:42:47 ERROR 404: Not Found.

dpkg: error: cannot access archive 'google-chrome-stable_90.0.4430.212-1_amd64.deb': No such file or directory
--2025-04-26 03:42:47--  https://chromedriver.storage.googleapis.com/90.0.4430.24/chromedriver_linux64.zip
Resolving chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)... 108.177.12.207, 108.177.11.207, 74.125.26.207, ...
Connecting to chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)|108.177.12.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5795569 (5.5M) [application/zip]
Saving to: ‘chromedriv

In [ ]:
# Update Your Environment PATH

import os
os.environ["PATH"] += ":/usr/bin/"

In [ ]:
import time, datetime
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
# Adjust Your Selenium Setup

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# Configure Chrome options
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

The following cell runs the web crawler.

In [ ]:
# The url where the data will be collected from.
url = "https://www.theguardian.com/us"

# Initialize WebDriver
bot = webdriver.Chrome(options=options)

# Input the targeting url to the bot, and the bot will load data from the url.
bot.get(url)

# An array to store all the article paths. If an article path has been crawled, it would not be stored to the data frame.
article_links = []
# An array to store the retrieved article titles.
results = []


# variable i indicates the number of times that scrolls down a web page. In practice, you might want to develop different
# interaction approach to load and view the web pages.

for i in range(1):

    # Create a document object model (DOM) from the raw source of the crawled web page.
    # Since you are processing a html page, 'html.parser' is chosen.
    soup = BeautifulSoup(bot.page_source, 'html.parser')

    # Capture all the article title items using find_all or findAll method.
    # To view the information of the html elements you want to collect, you need to inspect the raw source using Chrome Inspector.
    # To test whether you find the right html elements, you can use the pycharm debugger to examine the returned data.
    # May need to double check or adjust the class before running, in case it has updated.
    articles = soup.find_all('a', class_="dcr-2yd10d")

    # iterate and process each video entry.
    for article in articles:

        # I prefer use the "try-except" statement to enable the program run without pausing due to unexpected errors.
        # try:
            # video_url = video.find("a", class_="yt-simple-endpoint inline-block style-scope ytd-thumbnail").attrs["href"]
            # user_url = video.find("a", class_="yt-simple-endpoint style-scope yt-formatted-string").attrs["href"]
            # username = video.find("a", class_="yt-simple-endpoint style-scope yt-formatted-string").text
            # title = video.find("yt-formatted-string", class_="style-scope ytd-video-renderer").text
            # view_num = video.find_all("span", class_="style-scope ytd-video-meta-block")[0].text.replace(" views", "")
            # created_at = video.find_all("span", class_="style-scope ytd-video-meta-block")[0].text.replace(" ago", "")
            # shortdesc = video.find("yt-formatted-string", id="description-text").text
            # collected_at = datetime.datetime.now()

            article_text = article.attrs["aria-label"]
            article_link = article.attrs["href"]
            collected_at = datetime.datetime.now()

            # create a row in the dict format.
            row = {'article_text': article_text,
                    'article_link': article_link,
                    'collected_at': collected_at}

            # if an article path has been added, this article title would not be inserted to the results array,
            # otherwise, this title will be inserted.
            if article_link in article_links:
                print("this article has already been added.")
            else:
                print(row)
                article_links.append(article)
                results.append(row)
        # except:
        #     pass

    # it is very important to enable the bot take some rest, and then resume to work.
    time.sleep(5)

    # Let the bot scrolls down to the bottom of the content element, most of the time the bot needs to scroll down to the bottom of the page.
    # like this statement: bot.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    bot.execute_script('window.scrollTo(0,  document.body.scrollHeight);')

# terminate the bot object.
bot.close()

# Store the results as a pandas dataframe
df = pd.DataFrame(results)

# notify the completion of the crawling in the console.
print("the crawling task is finished.")

{'article_text': 'FBI arrests Wisconsin judge and accuses her of obstructing immigration officials', 'article_link': '/us-news/2025/apr/25/judge-hannah-dugan-milwaukee-arrest', 'collected_at': datetime.datetime(2025, 4, 26, 4, 50, 25, 198674)}
{'article_text': 'Virginia Giuffre, who accused Jeffrey Epstein of sexual abuse, dies aged 41', 'article_link': '/us-news/2025/apr/26/virginia-giuffre-suicide-dead-aged-41', 'collected_at': datetime.datetime(2025, 4, 26, 4, 50, 25, 199112)}
{'article_text': 'Son of CIA deputy director was killed while fighting for Russia, report says', 'article_link': '/us-news/2025/apr/25/michael-alexander-gloss-cia-russia', 'collected_at': datetime.datetime(2025, 4, 26, 4, 50, 25, 199158)}
{'article_text': 'US restoring legal status of hundreds of students after abruptly revoking visas', 'article_link': '/us-news/2025/apr/25/student-visa-legal-status-restored', 'collected_at': datetime.datetime(2025, 4, 26, 4, 50, 25, 199175)}
{'article_text': 'Royals and refug

In [ ]:
# Output the dataframe to confirm that data was entered as intended
df

,article_text,article_link,collected_at
0,FBI arrests Wisconsin judge and accuses her of...,/us-news/2025/apr/25/judge-hannah-dugan-milwau...,2025-04-26 04:50:25.198674
1,"Virginia Giuffre, who accused Jeffrey Epstein ...",/us-news/2025/apr/26/virginia-giuffre-suicide-...,2025-04-26 04:50:25.199112
2,Son of CIA deputy director was killed while fi...,/us-news/2025/apr/25/michael-alexander-gloss-c...,2025-04-26 04:50:25.199158
3,US restoring legal status of hundreds of stude...,/us-news/2025/apr/25/student-visa-legal-status...,2025-04-26 04:50:25.199175
4,Royals and refugees to come together in Rome f...,/world/2025/apr/26/royals-and-refugees-to-come...,2025-04-26 04:50:25.199191
...,...,...,...
106,"Wild ponies, a playful jaguar and penguin chic...",/environment/gallery/2025/apr/25/week-in-wildl...,2025-04-26 04:50:25.201881
107,Shot from the hip! A street level view of 1970...,/artanddesign/gallery/2025/apr/24/shot-from-th...,2025-04-26 04:50:25.201921
108,Holocaust Remembrance Day and mourners at the ...,/news/gallery/2025/apr/24/holocaust-remembranc...,2025-04-26 04:50:25.201960
109,Space craft and heavenly respite,/community/gallery/2025/apr/23/space-craft-and...,2025-04-26 04:50:25.201998


In [ ]:
# Save the dataframe as a csv file
df.to_csv('theguardianresults.csv', index=False)

# Go to this notebook's file directory to access and download the csv